<a href='https://ccpf.proscan.com/programs/queen-city-classic-chess-tournament/'><img src='https://ccpf.proscan.com/wp-content/themes/proscanfund/library/images/sidebar-queen-city-classic-chess-tournament.png'/></a>

### On 3 March 2018, my child competed with close to 700 other students in the [Queen City Classic Chess Tournament](https://ccpf.proscan.com/programs/queen-city-classic-chess-tournament/).  It was a long day, but my boy and his school did well and seemed to enjoy themselves.  

### The organizers graciously [publish the tournament results](https://ccpf.proscan.com/programs/queen-city-classic-chess-tournament/final-results/), albeit in PDF format.  Nevertheless, I thought this might make for a good opportunity to do a little bit of data analysis: what schools were most represented, grades, and so forth.  

### The analysis will be done over two notebooks.  The first, this one, will focus on downloading, parsing the PDFs, performing some rudimentary cleaning, and then serializing the data into usable CSV files.  [The second](./analyze_tournament_data.ipynb) will do the analysis work. 

### Step 1: Load packages

In [3]:
import requests
from bs4 import BeautifulSoup
import lxml
import PyPDF2  # installation instructions here: https://anaconda.org/conda-forge/pypdf2
import pandas as pd
import numpy as np
import re
import csv

### Step 2: Scrape the tournament results page containing the links to all the individual result documents

In [4]:
result = requests.get("https://ccpf.proscan.com/programs/queen-city-classic-chess-tournament/final-results/")
soup = BeautifulSoup(result.content, "lxml")

### Step 3: Grab the links to all the individual results documents

In [5]:
pdf_list = []
results_section = soup.find("section", class_="entry-content cf")
pdf_links = results_section.find_all("a")
for pdf_link in pdf_links:
    title = pdf_link.text
    link = pdf_link.attrs['href']
    pdf_list.append([title, link])

### Step 4: Let's go ahead and save those result documents to disk

In [14]:
for p in pdf_list:
    # save pdf to disk
    r = requests.get(p[1])
    pdf_file = p[1].split('/')[-1]
    open("./data/" + pdf_file, 'wb').write(r.content)

### Step 5: For reference, let's also get the player lists and save them to disk

In [11]:
# scrape the players list web page
pl_result = requests.get("https://ccpf.proscan.com/programs/queen-city-classic-chess-tournament/player-list/")
pl_soup = BeautifulSoup(pl_result.content, "lxml")

In [12]:
# grab all the hyperlinks to the various available pdfs that list the players
pl_pdf_list = []
results_section = pl_soup.find("section", class_="entry-content cf")
pl_pdf_links = results_section.find_all("a")
for pl_pdf_link in pl_pdf_links:
    title = pl_pdf_link.text
    link = pl_pdf_link.attrs['href']
    pl_pdf_list.append([title, link])

In [67]:
# now, download the pdfs and save them locally
for p in pl_pdf_list:
    # save pdf to disk
    r = requests.get(p[1])
    pdf_file = p[1].split('/')[-1]
    open("./data/" + pdf_file, 'wb').write(r.content)

### Step 6: Parse the downloaded PDFs

### So, here's my first problem: even though I've used [PyPDF2](https://pythonhosted.org/PyPDF2/) successfully in the past, the extractText method of PyPDF2 seems unable to parse these particular PDFs.  [The documentation says](https://pythonhosted.org/PyPDF2/PageObject.html#PyPDF2.pdf.PageObject):
 <blockquote>This [the extractText method] works well for some PDF files, but poorly for others, depending on the generator used.</blockquote>

### So, I guess, PyPDF2 won't work for me here.  Alright, what's Plan B?

In [ ]:
# My attempt to parse these PDFs with PyPDF2.  All I got was a bunch of blank text.

# def load_pdf_into_dataframe(pdf_file, title):
#     with open(pdf_file, "rb") as f:
#         pdf_reader = PyPDF2.PdfFileReader(f, strict=False)
        
#         for i in range(pdf_reader.numPages):
#             # iterate through the pages in the pdf
#             pdf_page = pdf_reader.getPage(i)
#             print(pdf_page.extractText())

        
# df = load_pdf_into_dataframe("./data/n1FINALIndiviual.pdf", "some title")


### For Plan B, let's see if the [Xpdf tools](https://www.xpdfreader.com/index.html) can help us parse the PDF files.

#### First, let's use the pdftotext utility to convert all the PDF documents to TXT documents that we can hopefully later parse.

In [70]:
pdftotext = '"C:\\Program Files (x86)\\xpdf-tools-win-4.00\\bin64\\pdftotext.exe"'  # path to the util on my hard drive

# loop through all the PDFs I downloaded and run the pdftotext utility against each of them
for p in (pdf_list + pl_pdf_list):
    pdf_path = './data/' + p[1].split('/')[-1]
    txt_path = pdf_path.replace('.pdf', '.txt')
    ! {pdftotext} -table {pdf_path} {txt_path}

#### Now, to pull the data into objects I can actually work with, numpy has this really interesting method [fromregex](https://docs.scipy.org/doc/numpy-1.13.0/reference/generated/numpy.fromregex.html).  If I'm clever enough to construct a regex that can match on each field of the data, the fromregex method can pull the data into a numpy array.  From my research, there appear to be three different table formats at work with this data:
<ol>
<li>A schema for Individual, non-ranked players</li>
<li>A schema for Individual, ranked players</li>
<li>And schema for team results, regardless of whether the players were ranked or not</li>
</ol>
#### So, with [a lot of testing](https://regex101.com/), I have devised three regular expressions for each of the three file schemas.  Now, I'll loop through the files and finally get some consolidated lists of this data.

#### Quick update: I ran into a few problems with numpy's [fromregex](https://docs.scipy.org/doc/numpy-1.13.0/reference/generated/numpy.fromregex.html) method.  Most of the records parsed just find, but some garbage data still found its way into the lists; so, I'll just parse the files with the tried-and-true [re](https://docs.python.org/3/library/re.html) package.

In [8]:
re_ind_nr = r'\s+([0-9]+)\s+(.+?(?=\s{2}))\s{2,}(.+?(?=\s{2}))\s+(.+?(?=\s{2}))\s+(.+?(?=\s{2}))\s+(.+?(?=\s{2}))\s+(.+?(?=\s{2}))\s+(.+?(?=\s{2}))\s+(.+?(?=\s{2}))\s+(.+?(?=\s{2}))\s+(.+?(?=\s{2}))\s+(.+)'
re_ind_r = r'\s+([0-9]+)\s+(.+?(?=\s{2}))\s{2,}(.+?(?=\s{2}))\s+([0-9 ]+?(?=\s{2}))\s+(.+?(?=\s{2}))\s+(.+?(?=\s{2}))\s+(.+?(?=\s{2}))\s+(.+?(?=\s{2}))\s+(.+?(?=\s{2}))\s+(.+?(?=\s{2}))\s+(.+?(?=\s{2}))\s+(.+?(?=\s{2}))\s+(.+)'
re_team = r'([0-9]+)\s+(.+?(?=\s{2}))\s+(.+?(?=\s{2}))\s+(.+?(?=\s{2}))\s+(.+?(?=\s{2}))\s+(.+?(?=\s{2}))\s+(.+?(?=\s{2}))\s+(.+)'
# the regex still let's some garbage rows come through like headers and footers.  use this list to weed the garbage out
elems_of_rows_to_remove = ['Score', 'Rnd1', 'Code', 'TBrk1']
ind_nr_list = []
ind_r_list = []
team_list = []

# iterate through the list of result files I downloaded.  The PDFs fall into one of three categories: team results, 
# ranked player results, or non-ranked player results.  The file names follow a loose convention: if "team" or "tm"
# is in the file name, that file is a list of team results.  If a file name starts with "n", that file represents
# results of non-ranked players.  All the rest are results of ranked players.
for p in pdf_list:
    title = p[0]
    txt_file = './data/{0}'.format(p[1].split('/')[-1].replace('.pdf', '.txt'))
    with open(txt_file, 'r') as f:
        t = f.read()
        if 'team' in title.lower() or 'tm' in title.lower():
            l = re.findall(re_team, t)
            l = [[title] + list(r) for r in l if not any(i in r for i in elems_of_rows_to_remove)]
            [team_list.append(r) for r in l]
        elif title.lower().startswith('n'):
            l = re.findall(re_ind_nr, t)
            l = [[title] + list(r) for r in l if not any(i in r for i in elems_of_rows_to_remove)]
            [ind_nr_list.append(r) for r in l]
        else:
            l = re.findall(re_ind_r, t)
            l = [[title] + list(r) for r in l if not any(i in r for i in elems_of_rows_to_remove)]
            [ind_r_list.append(r) for r in l]
        


[('2018', '-- Kindergarten NonRated', 'Team Standings, Page 1', 'Plc', 'Code', 'Name (Players:Top 3 used)', 'Score', 'Med   Solk  SBx2  Cum'), ('1', 'Dragon', 'Dragon Chess Center (3)', '10.0', '20.5', '35.5', '33.0', '29.5'), ('2', 'DetCCC', 'Detroit City Chess Club (2)', '1.5', '14.5', '23.0', '1.5', '4.0')]


[('2018', '-- Grade 2 NonRated', 'Team Standings,', 'Page', '1', 'Plc', 'Code', 'Name (Players:Top 3            used)                                 Score  Med   Solk  SBx2  Cum'), ('1', 'Dragon', 'Dragon Chess Center (3)', '12.5', '28.0', '47.0', '72.5', '40.0'), ('2', 'DetCCC', 'Detroit City Chess Club (5)', '9.0', '23.0', '40.0', '34.0', '28.5'), ('3', 'LincEl', 'Lincoln Elementary (5)', '7.5', '15.5', '27.5', '19.5', '16.0'), ('4', 'StIgna', 'St. Ignatius (2)', '4.5', '17.5', '29.5', '23.5', '15.5')]


[('2018', '-- K-12 Open', 'Team Standings, Page', '1', 'Plc', 'Code', 'Name (Players:Top 3 used)', 'Score  Med   Solk  SBx2  Cum'), ('1', 'DetCCC', 'Detroit City Chess Club 

#### Success!  Let's take a quick gander at the shape of the three lists.

In [53]:
print('For the teams list, there are {0} records.  The min record size is {1} and max is {2}.'.
      format(len(team_list), len(max(team_list, key=len)), len(min(team_list, key=len))))

print('For the Non-Ranked list, there are {0} records.  The min record size is {1} and max is {2}.'.
      format(len(ind_nr_list), len(max(ind_nr_list, key=len)), len(min(ind_nr_list, key=len))))

print('For the Ranked list, there are {0} records.  The min record size is {1} and max is {2}.'.
      format(len(ind_r_list), len(max(ind_r_list, key=len)), len(min(ind_r_list, key=len))))

For the teams list, there are 134 records.  The min record size is 9 and max is 9.
For the Non-Ranked list, there are 468 records.  The min record size is 13 and max is 13.
For the Ranked list, there are 208 records.  The min record size is 14 and max is 14.


#### Now, let's also parse the players list just in case I need them later.  Looking at the two player [list](https://ccpf.proscan.com/wp-content/uploads/2018/03/Player-list-as-of-03.02.18_by-player-name.pdf) [files](https://ccpf.proscan.com/wp-content/uploads/2018/03/Player-list-as-of-03.02.18_by-team-name.pdf) on the site, you can see that they both contain the same data, they're just sorted differently.  So, I'm just going to parse one of the files.

In [19]:
re_players = r'(.+?(?=\s{2}))\s{2,}(.+?(?=\s{2}))\s{2,}(.+?(?=\s{2}))\s{2,}(.+?(?=\s{2}))\s{2,}(.+?(?=\s{2}))\s{2,}(.+?(?=\s{2}))'
player_elems_to_remove = ['First Name']
players_list = []

# added on 17 Mar 2018, as the players list page now seems to be taken down
pl_pdf_list = [['Player list as of 03.02.18 at noon_by player name', 'Player-list-as-of-03.02.18_by-player-name.pdf']]
title = pl_pdf_list[0][0]

txt_file = './data/{0}'.format(pl_pdf_list[0][1].split('/')[-1].replace('.pdf', '.txt'))
with open(txt_file, 'r') as f:
    t = f.read()
    l = re.findall(re_players, t)
    test = re.findall(re_players, t)
    l = [[title] + list(r) for r in l if not any(i in r for i in player_elems_to_remove)]
    [players_list.append(r) for r in l]

In [63]:
print('For the Players list, there are {0} records.  The min record size is {1} and max is {2}.'.
      format(len(players_list), len(max(players_list, key=len)), len(min(players_list, key=len))))

print('\nDid you catch that?  {0} students competed in this tournament!'.format(len(players_list)))

For the Players list, there are 699 records.  The min record size is 7 and max is 7.

Did you catch that?  699 students competed in this tournament!


### Step 7: Phew!  It took a lot of work to parse these PDF files.  At this point, I'll just save these lists to CSV and save the analysis for [another notebook](./analyze_tournament_data.ipynb).

In [70]:
# save the team results to disk
with open("./data/queen_city_2018_team_results.csv", "w", newline='') as f:
    writer = csv.writer(f)
    writer.writerows(team_list)
    
# save the ranked player results to disk
with open("./data/queen_city_2018_ranked_player_results.csv", "w", newline='') as f:
    writer = csv.writer(f)
    writer.writerows(ind_r_list)
    
# save the non-ranked player results to disk
with open("./data/queen_city_2018_non_ranked_player_results.csv", "w", newline='') as f:
    writer = csv.writer(f)
    writer.writerows(ind_nr_list)
    
# save the player list to disk
with open("./data/queen_city_2018_registered_players.csv", "w", newline='') as f:
    writer = csv.writer(f)
    writer.writerows(players_list)